In [54]:
import sys
import collections
from os import listdir, path
from pyknp import Jumanpp
from gensim import models
from gensim.models.doc2vec import LabeledSentence

In [55]:
def corpus_files(): #学習対象のファイルのパス一覧を取得
    dirs = [path.join('./tweets/', x)
            for x in listdir('./tweets/') if not x.endswith('.txt')]
    docs = [path.join(x, y)
            for x in dirs for y in listdir(x)]]
    return docs

In [56]:
def read_document(path): #withを用いてファイルを読み込む with中のretrunでも終了処理される
    with open(path, 'r') as f:
        return f.read()

In [57]:
def split_into_words(text):　#Juman++を用いて解析する
    result = Jumanpp().analysis(text)
    return [mrph.midasi for mrph in result.mrph_list()]

In [58]:
def doc_to_sentence(doc, name): #上記関数の結果にタグ付け
    words = split_into_words(doc)
    return LabeledSentence(words=words, tags=[name])

In [59]:
def corpus_to_sentences(corpus):　#ファイルパスのリストを全部読み取って全て形態素解析する
    docs   = [read_document(x) for x in corpus]　#ここでファイル単位で格納
    for idx, (doc, name) in enumerate(zip(docs, corpus)):
        sys.stdout.write('\r前処理中 {}/{} {}'.format(idx, len(corpus),name))　#画面表示用　画面表示があると体感時間が変わるらしい
        yield doc_to_sentence(doc, name)

In [60]:
corpus = corpus_files()
sentences = corpus_to_sentences(corpus)

In [61]:
model = models.Doc2Vec(size=100,    #次元数 ドキュメントごとの特徴次元を決定する
                       iter=20,     #反復学習回数　多すぎると過学習&時間がかかりすぎるかと思いこの値　
                       alpha=0.025, #学習率 min_alphaに向かって線形で落ちていくのでmin_alphaを設定すべきかも
                       min_count=5, #学習に入れる単語の最低出現頻度 1だと支離滅裂になりすぎるかと思い5回
                      workers=4)    #並列実行数
model.build_vocab(sentences)        #単語登録

前処理中 7492/7493 ./tweets/onigirichan/onigirichan999.txt

In [62]:
print('\n訓練開始')
for epoch in range(30): #ここも反復学習のためのfor文　今にして思えば不要
    print('Epoch: {}'.format(epoch + 1))　#これも体感時間削減のための画面表示
    model.train(sentences,total_examples = len(corpus),epochs = epoch) #実際の学習を行う関数 ドキュメント本体、ドキュメントの数、学習回数を指定


訓練開始
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
Epoch: 10
Epoch: 11
Epoch: 12
Epoch: 13
Epoch: 14
Epoch: 15
Epoch: 16
Epoch: 17
Epoch: 18
Epoch: 19
Epoch: 20
Epoch: 21
Epoch: 22
Epoch: 23
Epoch: 24
Epoch: 25
Epoch: 26
Epoch: 27
Epoch: 28
Epoch: 29
Epoch: 30


In [63]:
model.save('doc2vec.model') #モデルの保存
model = models.Doc2Vec.load('doc2vec.model')　#モデルの読み込み

In [64]:
result = model.docvecs.most_similar('./tweets/onigirichan/onigirichan1.txt') #指定したドキュメントと類似度の高いドキュメント10個を学習データから引っ張り出す
#result = model.most_similar(positive=["愛"]) #doc2vecのモデルはword2vecも使えるので単語単位でも検索可能
print(result)

[('./tweets/kuzugate/kuzugate1765.txt', 0.3781484067440033), ('./tweets/onigirichan/onigirichan90.txt', 0.3621360659599304), ('./tweets/kuzugate/kuzugate726.txt', 0.3575366735458374), ('./tweets/kuzugate/kuzugate969.txt', 0.3327537477016449), ('./tweets/b1cute/b1cute2074.txt', 0.33220329880714417), ('./tweets/b1cute/b1cute1781.txt', 0.3313763439655304), ('./tweets/kuzugate/kuzugate1424.txt', 0.32063692808151245), ('./tweets/kuzugate/kuzugate1618.txt', 0.317220538854599), ('./tweets/kuzugate/kuzugate211.txt', 0.30157965421676636), ('./tweets/onigirichan/onigirichan1118.txt', 0.30070436000823975)]
